# 简单 RAG 中的上下文块头 （CCH）

检索增强生成 （RAG） 通过在生成响应之前检索相关的外部知识来提高语言模型的事实准确性。但是，标准分块通常会丢失重要的上下文，从而使检索效率降低。

上下文块标头 （CCH） 通过在嵌入之前为每个块预置高级上下文（如文档标题或章节标题）来增强 RAG。这可以提高检索质量并防止脱离上下文的响应。

### 步骤：
1. 数据接入：加载并预处理文本数据。
2. 使用上下文标题进行分块：提取章节标题并将其添加到块的前面。
3. 嵌入创建：将上下文增强的块转换为数字表示。
4. 语义搜索：根据用户查询检索相关块。
5. 响应生成：使用语言模型从检索到的文本中生成响应。
6. 评估：使用评分系统评估响应准确性。

In [1]:
import os
import numpy as np
import json
from openai import OpenAI
import fitz
from tqdm import tqdm

In [2]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

client = OpenAI(
    base_url="https://api.siliconflow.cn/v1/",
    api_key=os.getenv("OPENAI_API_KEY"))

# 根据上下文头来切分块

In [3]:
def generate_header_chunks(chunk, model="Qwen/Qwen3-8B"):
    system_prompt = "Generate a concise and informative title for the given text."
    response = client.chat.completions.create(
        model=model,
        temperature=0.0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Text: {chunk}"}
        ]
    )
    return response.choices[0].message.content.strip()
